### **WEATHER_API**

In [1]:
# from datetime import datetime, timedelta, timezone
# from geopy.geocoders import Nominatim
# import math
# from dotenv import load_dotenv
# import os
# import requests

# def get_base_datetime():

#     kst = timezone(timedelta(hours = 9))
#     now = datetime.now(timezone.utc).astimezone(kst)

#     available_hours = [2, 5, 8, 11, 14, 17, 20, 23]

#     current_hour = now.hour

#     base_hour = max([h for h in available_hours if h <= current_hour], default = 23)

#     if current_hour < 2:
#         now = now - timedelta(days = 1)
#         base_date = now.strftime('%Y%m%d')

#     else:
#         base_date = now.strftime('%Y%m%d')

#     base_time = f'{base_hour:02}00'

#     return base_date, base_time

# def get_lat_long_from_address(address):

#     geolocator = Nominatim(user_agent = "weather-geocoder")

#     location = geolocator.geocode(address)

#     if not location:
#         raise ValueError("주소를 찾을 수 없습니다.")
    
#     return location.latitude, location.longitude

# def latlong_to_xy(lat, long):

#     RE = 6371.00877
#     GRID = 5.0
#     SLAT1 = 30.0
#     SLAT2 = 60.0
#     OLONG = 126.0
#     OLAT = 38.0
#     XO = 43
#     YO = 136
#     DEGRAD = math.pi / 180.0

#     re = RE / GRID
#     slat1 = SLAT1 * DEGRAD
#     slat2 = SLAT2 * DEGRAD
#     olong = OLONG * DEGRAD
#     olat = OLAT * DEGRAD

#     sn = math.tan(math.pi * 0.25 + slat2 * 0.5) / math.tan(math.pi * 0.25 + slat1 * 0.5)
#     sn = math.log(math.cos(slat1) / math.cos(slat2)) / math.log(sn)
#     sf = math.tan(math.pi * 0.25 + slat1 * 0.5)
#     sf = math.pow(sf, sn) * math.cos(slat1) / sn
#     ro = math.tan(math.pi * 0.25 + olat * 0.5)
#     ro = re * sf / math.pow(ro, sn)

#     ra = math.tan(math.pi * 0.25 + lat * DEGRAD * 0.5)
#     ra = re * sf / math.pow(ra, sn)
#     theta = long * DEGRAD - olong
#     theta = (theta + math.pi) % (2 * math.pi) - math.pi
#     theta *= sn

#     x = ra * math.sin(theta) + XO + 0.5
#     y = ro - ra * math.cos(theta) + YO + 0.5
    
#     return int(x), int(y)

# load_dotenv()

# weather_api_key = os.getenv('WEATHER_API_KEY')

# base_date, base_time = get_base_datetime()

# user_input_location = input("지역을 입력하세요. (예 : 대전광역시 서구)")
# lat, long = get_lat_long_from_address(user_input_location)
# nx, ny = latlong_to_xy(lat, long)

# url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'
# params = {'serviceKey' : weather_api_key, 'pageNo' : '1', 'numOfRows' : '10', 'dataType' : 'JSON', 'base_date' : base_date, 'base_time' : base_time, 'nx' : nx, 'ny' : ny}

# response = requests.get(url, params = params)

# data = response.json()

##### **Chroma; grid_db**

In [2]:
from dotenv import load_dotenv
import pandas as pd
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

load_dotenv()

df = pd.read_excel("data/격자_위경도.xlsx")
df = df.fillna('')

documents = []

for i, row in df.iterrows():

    location = f"{row['1단계']} {row['2단계']} {row['3단계']}".strip()

    content = f"{location}의 격자 좌표는 X = {int(row['격자 X'])}, Y = {int(row['격자 Y'])}입니다."

    metadata = {
        "location" : location,
        "x" : int(row['격자 X']),
        "y" : int(row['격자 Y'])
    }

    doc = Document(page_content = content, metadata = metadata)

    documents.append(doc)

embedding = OpenAIEmbeddings(model = "text-embedding-3-large")

vectorstore = Chroma.from_documents(
    documents = documents,
    embedding = embedding,
    persist_directory = "./grid_db",
    collection_name = "g_db"
)

In [3]:
from datetime import datetime, timedelta, timezone
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import requests

def get_base_datetime():

    kst = timezone(timedelta(hours = 9))
    now = datetime.now(timezone.utc).astimezone(kst)

    available_hours = [2, 5, 8, 11, 14, 17, 20, 23]

    current_hour = now.hour

    base_hour = max([h for h in available_hours if h <= current_hour], default = 23)

    if current_hour < 2:
        now -= timedelta(days = 1)
    
    base_date = now.strftime('%Y%m%d')
    base_time = f'{base_hour:02}00'

    return base_date, base_time

# def get_grid_from_address(address):

#     parts = address.split()

#     grid_df = pd.read_excel("data/격자_위경도.xlsx")

#     filter = grid_df.copy()

#     if len(parts) >= 1:
#         filter = filter[filter['1단계'].str.contains(parts[0], na = False)]

#     if len(parts) >= 2:
#         filter = filter[filter['2단계'].str.contains(parts[1], na = False)]

#     if len(parts) >= 3:
#         filter = filter[filter['3단계'].fillna('').str.contains(parts[2], na = False)]

#     if filter.empty:
#         raise ValueError("입력한 주소에 해당하는 격자 정보를 찾을 수 없습니다.")
    
#     row = filter.iloc[0]

#     return int(row['격자 X']), int(row['격자 Y'])

def get_grid_from_address(address):

    vectorstore = Chroma(
        persist_directory = "./grid_db",
        embedding_function = OpenAIEmbeddings(model = "text-embedding-3-large"),
        collection_name = "g_db"
    )

    result = vectorstore.similarity_search(query = address, k = 1)

    if not result:
        raise ValueError("해당 주소에 대한 격자 정보를 찾을 수 없습니다.")
    
    result_doc = result[0]

    x = result_doc.metadata['x']
    y = result_doc.metadata['y']

    return x, y

def fetch_weather_data(address):

    load_dotenv()

    weather_api_key = os.getenv('WEATHER_API_KEY')

    base_date, base_time = get_base_datetime()

    nx, ny = get_grid_from_address(address)

    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'
    params = {
        'serviceKey' : weather_api_key,
        'pageNo' : '1',
        'numOfRows' : '1000',
        'dataType' : 'JSON',
        'base_date' : base_date,
        'base_time' : base_time,
        'nx' : nx,
        'ny' : ny
    }

    response = requests.get(url, params = params)

    data = response.json()

    return data

def extract_weather_fields(data):

    items = data['response']['body']['items']['item']

    summaries = []

    for item in items:

        summary = {
            "category" : item["category"],
            "fcstDate" : item["fcstDate"],
            "fcstTime" : item["fcstTime"],
            "fcstValue" : item["fcstValue"]
        }

        summaries.append(summary)

    return summaries

CATEGORY_KOREAN = {
    'POP': '강수 확률 (%)',
    'PTY': '강수 형태',
    'PCP': '1시간 강수량 (mm)',
    'REH': '습도 (%)',
    'SNO': '1시간 신적설 (cm)',
    'SKY': '하늘 상태',
    'TMP': '1시간 기온 (℃)',
    'TMN': '일 최저 기온 (℃)',
    'TMX': '일 최고 기온 (℃)',
    'UUU': '풍속 (동서 성분) (m/s)',
    'VVV': '풍속 (남북 성분) (m/s)',
    'WAV': '파고 (M)',
    'VEC': '풍향 (deg)',
    'WSD': '풍속 (m/s)'
}

def format_weather_summaries(summaries):

    text_list = []

    for s in summaries:

        category_korean = CATEGORY_KOREAN.get(s['category'], s['category'])

        line = f"{s['fcstDate']} {s['fcstTime']} | {category_korean} : {s['fcstValue']}"

        text_list.append(line)

    weather_text = "\n".join(text_list)

    return weather_text

user_input_location = input("지역을 입력하세요. (예 : 대전광역시 서구)")

try:
    
    data = fetch_weather_data(user_input_location)
    summaries = extract_weather_fields(data)
    weather_text = format_weather_summaries(summaries)

    print(weather_text)

except ValueError as ve:
    print(ve)

except Exception as e:
    print(e)

20250718 1200 | 1시간 기온 (℃) : 27
20250718 1200 | 풍속 (동서 성분) (m/s) : 0.1
20250718 1200 | 풍속 (남북 성분) (m/s) : 2.8
20250718 1200 | 풍향 (deg) : 182
20250718 1200 | 풍속 (m/s) : 2.8
20250718 1200 | 하늘 상태 : 4
20250718 1200 | 강수 형태 : 0
20250718 1200 | 강수 확률 (%) : 30
20250718 1200 | 파고 (M) : -999
20250718 1200 | 1시간 강수량 (mm) : 강수없음
20250718 1200 | 습도 (%) : 85
20250718 1200 | 1시간 신적설 (cm) : 적설없음
20250718 1300 | 1시간 기온 (℃) : 27
20250718 1300 | 풍속 (동서 성분) (m/s) : 0.2
20250718 1300 | 풍속 (남북 성분) (m/s) : 1.9
20250718 1300 | 풍향 (deg) : 186
20250718 1300 | 풍속 (m/s) : 1.9
20250718 1300 | 하늘 상태 : 4
20250718 1300 | 강수 형태 : 1
20250718 1300 | 강수 확률 (%) : 60
20250718 1300 | 파고 (M) : -999
20250718 1300 | 1시간 강수량 (mm) : 1mm 미만
20250718 1300 | 습도 (%) : 90
20250718 1300 | 1시간 신적설 (cm) : 적설없음
20250718 1400 | 1시간 기온 (℃) : 29
20250718 1400 | 풍속 (동서 성분) (m/s) : -0.8
20250718 1400 | 풍속 (남북 성분) (m/s) : 3.8
20250718 1400 | 풍향 (deg) : 167
20250718 1400 | 풍속 (m/s) : 3.9
20250718 1400 | 하늘 상태 : 4
20250718 1400 | 강수 형태 : 0
202

### **SITOTOXISM_API**

In [4]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

sitotoxism_api_key = os.getenv('SITOTOXISM_API_KEY')

initial_region_url = f"http://openapi.foodsafetykorea.go.kr/api/{sitotoxism_api_key}/I2848/json/1/1"

initial_region_response = requests.get(initial_region_url)

initial_region_data = initial_region_response.json()

region_total_count = int(initial_region_data['I2848'].get('total_count', 0))

region_rows = []

batch_size = 1000

for region_start_idx in range(1, region_total_count + 1, batch_size):

    region_end_idx = min(region_start_idx + batch_size - 1, region_total_count)

    region_url = f"http://openapi.foodsafetykorea.go.kr/api/{sitotoxism_api_key}/I2848/json/{region_start_idx}/{region_end_idx}"
    
    region_response = requests.get(region_url)

    if region_response.status_code == 200:
        region_data = region_response.json()
        region_row = region_data['I2848'].get('row', [])
        region_rows.extend(region_row)

    else:
        break

In [5]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

sitotoxism_api_key = os.getenv('SITOTOXISM_API_KEY')

initial_facility_url = f"http://openapi.foodsafetykorea.go.kr/api/{sitotoxism_api_key}/I2849/json/1/1"

initial_facility_response = requests.get(initial_facility_url)

initial_facility_data = initial_facility_response.json()

facility_total_count = int(initial_facility_data['I2849'].get('total_count', 0))

facility_rows = []

batch_size = 1000

for facility_start_idx in range(1, facility_total_count + 1, batch_size):

    facility_end_idx = min(facility_start_idx + batch_size - 1, facility_total_count)

    facility_url = f"http://openapi.foodsafetykorea.go.kr/api/{sitotoxism_api_key}/I2849/json/{facility_start_idx}/{facility_end_idx}"
    
    facility_response = requests.get(facility_url)

    if facility_response.status_code == 200:
        facility_data = facility_response.json()
        facility_row = facility_data['I2849'].get('row', [])
        facility_rows.extend(facility_row)

    else:
        break

In [6]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

sitotoxism_api_key = os.getenv('SITOTOXISM_API_KEY')

initial_virus_url = f"http://openapi.foodsafetykorea.go.kr/api/{sitotoxism_api_key}/I2850/json/1/1"

initial_virus_response = requests.get(initial_virus_url)

initial_virus_data = initial_virus_response.json()

virus_total_count = int(initial_virus_data['I2850'].get('total_count', 0))

virus_rows = []

batch_size = 1000

for virus_start_idx in range(1, virus_total_count + 1, batch_size):

    virus_end_idx = min(virus_start_idx + batch_size - 1, virus_total_count)

    virus_url = f"http://openapi.foodsafetykorea.go.kr/api/{sitotoxism_api_key}/I2850/json/{virus_start_idx}/{virus_end_idx}"
    
    virus_response = requests.get(virus_url)

    if virus_response.status_code == 200:
        virus_data = virus_response.json()
        virus_row = virus_data['I2850'].get('row', [])
        virus_rows.extend(virus_row)

    else:
        break

##### **Chroma; sitotoxism_db**

In [7]:
from dotenv import load_dotenv
from langchain.schema import Document
import json
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

load_dotenv()

sitotoxism_data = []

# for source, rows in [("region", region_rows), ("facility", facility_rows), ("virus", virus_rows)]:
#     for idx, row in enumerate(rows):
#         doc = Document(
#             page_content = json.dumps(row, ensure_ascii = False),
#             metadata = {"source" : source, "index" : idx}
#         )
#         sitotoxism_data.append(doc)

for source, rows in [("region", region_rows), ("facility", facility_rows), ("virus", virus_rows)]:
    
    for row in rows:

        if source == "region":

            id = f"{row.get('OCCRNC_YEAR')}-{row.get('OCCRNC_MM')}-{row.get('OCCRNC_AREA')}"

            content = (
                f"{row.get('OCCRNC_YEAR', '?')}년 {row.get('OCCRNC_MM', '?')}월, "
                f"{row.get('OCCRNC_AREA', '?')} 지역에서 식중독이 {row.get('OCCRNC_CNT', '?')}건 발생했으며 "
                f"환자 수는 {row.get('PATNT_CNT', '?')}명이었습니다."
            )

        elif source == "facility":

            id = f"{row.get('OCCRNC_YEAR')}-{row.get('OCCRNC_MM')}-{row.get('OCCRNC_PLC')}"

            content = (
                f"{row.get('OCCRNC_YEAR', '?')}년 {row.get('OCCRNC_MM', '?')}월, "
                f"{row.get('OCCRNC_PLC', '?')} 시설에서 식중독이 {row.get('OCCRNC_CNT', '?')}건 발생했으며 "
                f"환자 수는 {row.get('PATNT_CNT', '?')}명이었습니다."
            )

        elif source == "virus":

            id = f"{row.get('OCCRNC_YEAR')}-{row.get('OCCRNC_MM')}-{row.get('OCCRNC_VIRS')}"

            content = (
                f"{row.get('OCCRNC_YEAR', '?')}년 {row.get('OCCRNC_MM', '?')}월, "
                f"{row.get('OCCRNC_VIRS', '?')} 바이러스로 인해 식중독이 {row.get('OCCRNC_CNT', '?')}건 발생했고 "
                f"환자 수는 {row.get('PATNT_CNT', '?')}명이었습니다."
            )

        else:
            id = "unknown"
            content = str(row)

        doc = Document(
            page_content = content,
            metadata = {"source" : source, "sitotoxism_id" : id}
        )

        sitotoxism_data.append(doc)

embedding = OpenAIEmbeddings(model = "text-embedding-3-large")

vectorstore = Chroma.from_documents(
    documents = sitotoxism_data,
    embedding = embedding,
    persist_directory = "./sitotoxism_db",
    collection_name = "s_db"
)

In [8]:
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

load_dotenv()

vectorstore = Chroma(
    persist_directory = "./sitotoxism_db",
    embedding_function = OpenAIEmbeddings(model = "text-embedding-3-large"),
    collection_name = "s_db"
)

vectorstore.get()

{'ids': ['808314c9-9001-4388-9e3f-8c56575ceb47',
  '1874df7d-1392-4550-b5b9-ca4a64ecaeb0',
  '08c7ca15-8ba2-47e6-8395-bf1be38c713c',
  'faf0742d-a43b-4152-b5e5-d44d06547109',
  'a5e07304-aeba-4d7d-be74-b0ea36854535',
  'c58eab2d-656e-4551-8ad3-cf72d89e9ec6',
  '7ca0872d-4eca-4225-9baf-1cad6f502aa9',
  '00949c3c-f232-4cdd-80b3-dac50b513350',
  'fa97881b-6614-4cd4-9a08-d8c012260fc2',
  'ec63634e-fdbf-4e8a-abda-47398836b3e4',
  'd24a4a8b-37b6-4cf6-8dd0-c9a9ea4f6a89',
  '7ac5b62d-b020-4b05-bae5-0195c3c9ee32',
  'd66b31e5-eb2e-4260-ad6d-8deca56d17e0',
  '467048c4-46a2-44c2-aa94-317c968c79dc',
  'b154b568-4ce5-45b7-8fa1-dd0548752764',
  '33850534-666e-4352-8b47-10963c31d404',
  '96527ee6-fd00-421f-8694-3b76099c111f',
  '78bbd969-22da-4ac4-be5a-1c4221b01cb7',
  '56c69641-3379-4239-baf6-e200aa44e90f',
  'c379e4c7-4538-49ec-8ddd-8d2d9d12ae67',
  'cbbae83c-a90b-4cbb-bbe9-0a40f824a3cd',
  'ffab9891-7092-47e2-b022-f008a268203f',
  '6ac45c09-e597-4a25-8c0f-63f867928336',
  '84708910-4cb3-4497-b5b6-

### **RISK DATA**

In [9]:
import os, time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

download_path = os.path.abspath("download")

os.makedirs(download_path, exist_ok = True)

for file in os.listdir(download_path):
    if file.endswith(".xlsx"):
        os.remove(os.path.join(download_path, file))

options = Options()

options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

options.add_experimental_option("prefs", {"download.default_directory" : download_path, "download.prompt_for_download" : False, "directory_upgrade" : True})

driver = webdriver.Chrome(options = options)

driver.get("https://poisonmap.mfds.go.kr/reference.do")

try:

    wait = WebDriverWait(driver, 10)

    download_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[onclick='downloadExcel()']")))

    driver.execute_script("arguments[0].click();", download_button)

except Exception as e:
    print(e)

def wait_for_download_complete(download_path, timeout = 30):

    second = 0

    while second < timeout:

        file = os.listdir(download_path)

        downloading = [f for f in file if f.endswith(".crdownload")]

        if not downloading and any(f.endswith(".xlsx") for f in file):
            return

        time.sleep(1)

        second += 1

    raise TimeoutError("제한 시간 초과")

wait_for_download_complete(download_path)

driver.quit()

def get_latest_excel_file(download_path):

    file = [f for f in os.listdir(download_path) if f.endswith(".xlsx")]

    if not file:
        raise FileNotFoundError("파일을 찾을 수 없습니다.")
    
    latest_file = max(
        [os.path.join(download_path, f) for f in file], key = os.path.getctime
    )

    return latest_file

file_path = get_latest_excel_file(download_path)

def read_excel_file(file_path):

    df = pd.read_excel(file_path)

    exclude_cols = ['14시 기준 온도', '14시 기준 습도', '14시 기준 강수량', '미세먼지']

    df = df.drop(columns = [col for col in exclude_cols if col in df.columns])

    df = df.rename(columns = {'예측지수' : '식중독_예측지수'})

    agg_dictionary = {
        col : ('mean' if col == '식중독_예측지수' else (lambda x : x.mode().iloc[0] if not x.mode().empty else x.iloc[0]))
        for col in df.columns if col not in ['시도', '시군구']
    }

    df = df.groupby(['시도', '시군구'], as_index = False).agg(agg_dictionary)

    display(df.head())

    return df

df = read_excel_file(file_path)

c:\big_data\sitotoxism_venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,시도,시군구,일자,식중독_예측지수,단계
0,강원특별자치도,강릉시,20250718,89.3,심각
1,강원특별자치도,고성군,20250718,92.0,심각
2,강원특별자치도,동해시,20250718,92.2,심각
3,강원특별자치도,삼척시,20250718,90.0,심각
4,강원특별자치도,속초시,20250718,90.1,심각


In [10]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model = "gpt-4o", temperature = 0)

df_preview = df.head().to_markdown(index = False)

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 데이터 분석가입니다. 사용자에게 친절하게 설명해 주세요."),
    ("human", "다음은 엑셀에서 추출한 데이터입니다.\n\n{data}\n\n이 데이터의 의미 있는 인사이트를 알려 줘.")
])

messages = prompt.format_messages(data = df_preview)

response = llm(messages)

print(response.content)

C:\Users\main\AppData\Local\Temp\ipykernel_11756\2554788503.py:15: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


이 데이터는 강원특별자치도의 여러 시군구에서 2025년 7월 18일에 예측된 식중독 지수와 그에 따른 단계 정보를 보여줍니다. 이 데이터를 통해 몇 가지 인사이트를 얻을 수 있습니다:

1. **식중독 위험 수준**: 모든 지역에서 식중독 예측지수가 89.3 이상으로, '심각' 단계에 해당합니다. 이는 해당 날짜에 강원특별자치도 전역에서 식중독 발생 위험이 매우 높다는 것을 의미합니다.

2. **지역별 비교**: 고성군과 동해시의 예측지수가 각각 92와 92.2로 가장 높습니다. 이는 이 두 지역이 다른 지역보다 식중독 발생 가능성이 더 높을 수 있음을 시사합니다.

3. **예방 조치 필요성**: '심각' 단계는 식중독 예방을 위한 즉각적인 조치가 필요함을 나타냅니다. 따라서 해당 지역의 보건 당국은 식중독 예방을 위한 캠페인, 식품 안전 점검, 교육 프로그램 등을 강화할 필요가 있습니다.

4. **날씨와의 연관성**: 식중독 지수는 일반적으로 기온, 습도 등과 같은 기상 조건과 밀접한 관련이 있습니다. 따라서 이 시기에 기상 조건이 식중독 발생에 유리한 환경을 제공했을 가능성이 있습니다.

이러한 인사이트를 바탕으로, 해당 지역의 주민들과 관련 기관은 식중독 예방을 위한 적절한 조치를 취할 수 있을 것입니다.
